In [1]:
!pip3 install police-api-client==1.0.1


  Using cached police-api-client-1.0.1.tar.gz (9.2 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for police-api-client: filename=police_api_client-1.0.1-py3-none-any.whl size=11202 sha256=33b6fa88a657d65e5579dd87949652a9cbdf2455350fd7124a450e51f367bfac
  Stored in directory: c:\users\x1 yoga\appdata\local\pip\cache\wheels\45\07\22\2f1ce0367f052649bf261fccfbb9b1c93f4c213a88873e3ac0
Successfully built police-api-client



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip3 install pandas
import json
!pip3 install matplotlib


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from police_api import PoliceAPI
api = PoliceAPI()

In [4]:
import requests
print(requests.get('https://data.police.uk/api/crime-last-updated').json())

{'date': '2025-02-01'}


In [5]:
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
from datetime import datetime

In [6]:
api.get_forces()
# [<Force> Avon and Somerset Constabulary, ..., <Force> Wiltshire Police]
leicestershire = api.get_force('city-of-london')

print(leicestershire)

<Force> City of London Police


In [7]:
leicestershire.neighbourhoods
neighbourhood = api.get_neighbourhood('city-of-london', 'ban')


In [8]:
crime = api.get_crime('ddf4c172d29569ab0cb667a346bcffad18f54a9bc3e0ae9694d2daf6738f068b')
crime

APIError: <unknown error code>

In [41]:
print(api.get_crimes_area(neighbourhood.boundary, date='2013-10'))

APIError: <unknown error code>

In [27]:
api.get_latest_date()
# u'2013-09'
api.get_crimes_area(city_centre.boundary)
# [<Crime> 26926242, ..., <Crime> 26925710]
len(api.get_crimes_area(city_centre.boundary, date='2013-08'))
# 810
crime = api.get_crimes_area(city_centre.boundary, date='2013-06')[0]
crime.outcomes
#[<Crime.Outcome> Under investigation, <Crime.Outcome> Suspect charged, <Crime.Outcome> Awaiting court outcome]

APIError: <unknown error code>

In [ ]:
def get_crime_data(force_id, year_month):
    url = f"https://data.police.uk/api/crimes-street/all-crime?force={force_id}&date={year_month}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises HTTPError for bad status codes
        return response.json()
    except requests.exceptions.JSONDecodeError:
        print(f"Empty/bad response for {year_month}. Possible invalid date or no data.")
        return []
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

# **1. Determine latest valid date**
latest_data = requests.get('https://data.police.uk/api/crime-last-updated').json()
latest_month = latest_data['date']
print(f"Latest available month: {latest_month}")

# 2. Get City of London data for latest month
force_id = 'city-of-london'
crimes = get_crime_data(force_id, latest_month)  # Use latest_month here


Latest available month: 2025-02
Request failed: 400 Client Error: BAD REQUEST for url: https://data.police.uk/api/crimes-street/all-crime?force=city-of-london&date=2025-02
No crime data available.


In [20]:
def get_available_months():
    """
    Returns a list of available YYYY-MM strings, sorted newest first.
    """
    url = "https://data.police.uk/api/crimes-street-dates"
    resp = requests.get(url)
    resp.raise_for_status()  # raises HTTPError for 4xx/5xx :contentReference[oaicite:2]{index=2}
    js = resp.json()
    # Each item is { "date": "YYYY-MM", ... }
    return [item["date"] for item in sorted(js, key=lambda x: x["date"], reverse=True)]


In [ ]:
def get_crime_data(force_id, year_month):
    url = f"https://data.police.uk/api/crimes-street/all-crime?force={force_id}&date={year_month}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException:
        # This covers HTTPError, ConnectionError, Timeout, etc.
        return []

force_id = "city-of-london"
available_months = get_available_months()
print("Trying months:", available_months[:3], "...")  # debug

# Iterate through the 12 most recent months (or however many you like)
for month in available_months[:24]:
    crimes = get_crime_data(force_id, month)
    if crimes:
        print(f"Found data for {month}!")
        break
else:
    raise RuntimeError("No crime data found in the last 12 months.")

# At this point, `crimes` contains the first non-empty result,
# and `month` holds the corresponding YYYY-MM string.


Trying months: ['2025-02', '2025-01', '2024-12'] ...


RuntimeError: No crime data found in the last 12 months.

In [18]:
# Error handling wrapper
def get_crime_data(force_id, year_month):
    url = f"https://data.police.uk/api/crimes-street/all-crime?force={force_id}&date={year_month}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises HTTPError for bad status codes
        return response.json()
    except requests.exceptions.JSONDecodeError:
        print(f"Empty/bad response for {year_month}. Possible invalid date or no data.")
        return []
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

# Get City of London data
force_id = 'city-of-london'
latest_data = requests.get('https://data.police.uk/api/crime-last-updated').json()
latest_month = latest_data['date']
print(type(latest_data['date']))

print(f"Latest available month: {latest_month}")
date = '2024-02-01' # Use older months
crimes = get_crime_data(force_id, date)

if not crimes:
    print("No data received. Check date validity (format YYYY-MM) and try older months.")
else:
    # Filter burglaries and create DataFrame
    burglaries = [c for c in crimes if c['category'] == 'burglary']
    df = pd.json_normalize(burglaries)
    
    # Display first 10 entries
    print("First 10 burglaries:")
    print(df[['month', 'location.street.name', 'location.latitude', 'location.longitude']].head(10))
    
    # Add time analysis
    if 'outcome_status.date' in df.columns:
        df['datetime'] = pd.to_datetime(df['outcome_status.date'])
        df['hour'] = df['datetime'].dt.hour
        df['is_weekend'] = df['datetime'].dt.dayofweek >= 5
    else:
        # Fallback to month-level analysis if no precise timestamps
        df['datetime'] = pd.to_datetime(df['month'])
        print("\nWarning: No hourly data available - using month-level analysis")

    # Visualization
    plt.figure(figsize=(15, 5))
    
    # Time analysis
    if 'hour' in df.columns:
        plt.subplot(1, 3, 1)
        df['hour'].value_counts().sort_index().plot(kind='bar')
        plt.title('Burglary Frequency by Hour')
    
    
    # Weekend analysis
    if 'is_weekend' in df.columns:
        plt.subplot(1, 3, 2)
        df['is_weekend'].value_counts().plot(kind='pie', autopct='%1.1f%%')
        plt.title('Weekend vs Weekday Burglaries')
    
    # LSOA Analysis
    plt.subplot(1, 3, 3)
    df['location.lsoa'].value_counts().head(10).plot(kind='barh')
    plt.title('Top 10 LSOA Areas')
    
    plt.tight_layout()
    plt.show()


<class 'str'>
Latest available month: 2025-02-01
Request failed: 404 Client Error: NOT FOUND for url: https://data.police.uk/api/crimes-street/all-crime?force=city-of-london&date=2024-02-01
No data received. Check date validity (format YYYY-MM) and try older months.
